In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df = pd.read_csv('data/KSI.csv')
df.shape

(16860, 57)

In [31]:
data = df[[ 'YEAR', 'DATE', 'TIME', 'HOUR', 'STREET1', 'STREET2', 'ROAD_CLASS', 'DISTRICT', 'LOCCOORD', 'ACCLOC', 'TRAFFCTL', 'VISIBILITY', 'LIGHT',
 'RDSFCOND', 'ACCLASS', 'IMPACTYPE', 'INVTYPE', 'INVAGE', 'INJURY', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 
 'PEDCOND',  'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK','TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER', 'SPEEDING', 'AG_DRIV', 'REDLIGHT',
  'ALCOHOL', 'DISABILITY', 'POLICE_DIVISION','NEIGHBOURHOOD']]


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 42 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   YEAR             16860 non-null  int64 
 1   DATE             16860 non-null  object
 2   TIME             16860 non-null  int64 
 3   HOUR             16860 non-null  int64 
 4   STREET1          16860 non-null  object
 5   STREET2          16860 non-null  object
 6   ROAD_CLASS       16860 non-null  object
 7   DISTRICT         16860 non-null  object
 8   LOCCOORD         16860 non-null  object
 9   ACCLOC           16860 non-null  object
 10  TRAFFCTL         16860 non-null  object
 11  VISIBILITY       16860 non-null  object
 12  LIGHT            16860 non-null  object
 13  RDSFCOND         16860 non-null  object
 14  ACCLASS          16860 non-null  object
 15  IMPACTYPE        16860 non-null  object
 16  INVTYPE          16860 non-null  object
 17  INVAGE           16860 non-null

In [33]:
data.describe()

,YEAR,TIME,HOUR
count,16860.000000,16860.000000,16860.000000
mean,2012.256168,1352.106821,13.241103
std,4.252797,629.726747,6.302679
min,2006.000000,0.000000,0.000000
25%,2009.000000,913.000000,9.000000
50%,2012.000000,1442.000000,14.000000
75%,2016.000000,1845.000000,18.000000
max,2020.000000,2359.000000,23.000000


### Exploratory Data Analysis

In [34]:
# Accident Class
round(df.ACCLASS.value_counts()/len(df) * 100, 2)

Non-Fatal Injury        86.36
Fatal                   13.62
Property Damage Only     0.01
Name: ACCLASS, dtype: float64

In [37]:
sns.countplot(data['ACCLASS'])

ValueError: could not convert string to float: 'Fatal'